In [45]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

from sklearn.preprocessing import RobustScaler

In [46]:
alarms = pd.read_csv('Alarms_of_the_sensors.csv', sep=None, engine='python')

In [47]:
alarms.head()

,time,sensor_id,alarm_type,severity,status,Assignee,atm_pm_1_0,atm_pm_2_5,co2,delta,ma,temperature
0,2025-11-10 16:19:48,Sensor_0057,Уровень CO2,Критический,Сброшенные неподтвержденные,2,0,1,709,38,1,27
1,2025-11-09 05:38:38,Sensor_0013,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,1144,27,0,30
2,2025-11-09 05:38:38,Sensor_0089,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,2474,28,0,27
3,2025-11-09 05:38:38,Sensor_0079,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,402,29,0,27
4,2025-11-09 05:38:38,Sensor_0068,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,606,25,0,27


In [49]:
print(alarms.columns)

Index(['time', 'sensor_id', 'alarm_type', 'severity', 'status', 'Assignee',
       'atm_pm_1_0', 'atm_pm_2_5', 'co2', 'delta', 'ma', 'temperature'],
      dtype='object')


In [50]:
alarms.head()

,time,sensor_id,alarm_type,severity,status,Assignee,atm_pm_1_0,atm_pm_2_5,co2,delta,ma,temperature
0,2025-11-10 16:19:48,Sensor_0057,Уровень CO2,Критический,Сброшенные неподтвержденные,2,0,1,709,38,1,27
1,2025-11-09 05:38:38,Sensor_0013,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,1144,27,0,30
2,2025-11-09 05:38:38,Sensor_0089,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,2474,28,0,27
3,2025-11-09 05:38:38,Sensor_0079,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,402,29,0,27
4,2025-11-09 05:38:38,Sensor_0068,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,606,25,0,27


In [51]:
alarms = alarms.drop(columns=['Assignee', 'delta'], errors='ignore')

In [52]:
columns_to_show = ['Timestamp', 'sensor_id', 'alarm_type', 'severity', 'status']
existing_columns = [col for col in columns_to_show if col in alarms.columns]
print(alarms[existing_columns].head())

     sensor_id         alarm_type     severity                       status
0  Sensor_0057        Уровень CO2  Критический  Сброшенные неподтвержденные
1  Sensor_0013  Потеря соединения  Критический  Сброшенные неподтвержденные
2  Sensor_0089  Потеря соединения  Критический  Сброшенные неподтвержденные
3  Sensor_0079  Потеря соединения  Критический  Сброшенные неподтвержденные
4  Sensor_0068  Потеря соединения  Критический  Сброшенные неподтвержденные


In [53]:
alarms['alarm_type'].unique()

array(['Уровень CO2', 'Потеря соединения', 'Курение',
       'Высокая концентрация CO2', 'Высокая температура',
       'Низкая влажность'], dtype=object)

In [54]:
alarms['severity'].unique()

array(['Критический'], dtype=object)

In [55]:
from datetime import datetime, timezone

In [56]:
alarms = pd.read_csv('Alarms_of_the_sensors.csv', sep=None, engine='python')

In [57]:
alarms.head()

,time,sensor_id,alarm_type,severity,status,Assignee,atm_pm_1_0,atm_pm_2_5,co2,delta,ma,temperature
0,2025-11-10 16:19:48,Sensor_0057,Уровень CO2,Критический,Сброшенные неподтвержденные,2,0,1,709,38,1,27
1,2025-11-09 05:38:38,Sensor_0013,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,1144,27,0,30
2,2025-11-09 05:38:38,Sensor_0089,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,2474,28,0,27
3,2025-11-09 05:38:38,Sensor_0079,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,402,29,0,27
4,2025-11-09 05:38:38,Sensor_0068,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,606,25,0,27


In [64]:
alarms['time'] = pd.to_datetime(alarms['time'], utc=True)

# Convert from its current timezone to UTC+5 if needed
# If you know the timestamps are already UTC+5 logically but in UTC format:
alarms['time'] = alarms['time'].dt.tz_convert('Etc/GMT-5')

# Then convert to pure UTC for database consistency
alarms['time_utc'] = alarms['time'].dt.tz_convert('UTC')

# Create UNIX timestamp column (seconds)
alarms['timestamp'] = alarms['time_utc'].astype('int64') // 10**9

alarms.head()

,time,sensor_id,alarm_type,severity,status,Assignee,atm_pm_1_0,atm_pm_2_5,co2,delta,ma,temperature,time_utc,timestamp
0,2025-11-10 16:19:48+05:00,Sensor_0057,Уровень CO2,Критический,Сброшенные неподтвержденные,2,0,1,709,38,1,27,2025-11-10 11:19:48+00:00,1762773588
1,2025-11-09 05:38:38+05:00,Sensor_0013,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,1144,27,0,30,2025-11-09 00:38:38+00:00,1762648718
2,2025-11-09 05:38:38+05:00,Sensor_0089,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,2474,28,0,27,2025-11-09 00:38:38+00:00,1762648718
3,2025-11-09 05:38:38+05:00,Sensor_0079,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,402,29,0,27,2025-11-09 00:38:38+00:00,1762648718
4,2025-11-09 05:38:38+05:00,Sensor_0068,Потеря соединения,Критический,Сброшенные неподтвержденные,0,0,0,606,25,0,27,2025-11-09 00:38:38+00:00,1762648718


In [66]:
sensor_ids = [
    "Sensor_0013",
    "Sensor_0039",
    "Sensor_0045",
    "Sensor_0057",
    "Sensor_0068",
    "Sensor_0079",
    "Sensor_0080",
    "Sensor_0089"
]

sensor_data = {}

for sensor in sensor_ids:
    filename = f"{sensor}_150_days.csv"
    df = pd.read_csv(filename, sep=None, engine="python")

    # Convert timestamp
    df['time'] = pd.to_datetime(df['time'], utc=True)
    df['time'] = df['time'].dt.tz_convert("Etc/GMT-5")  # Interpret as UTC+5
    df['time_utc'] = df['time'].dt.tz_convert("UTC")
    df['timestamp'] = df['time_utc'].astype("int64") // 10**9

    # Save into dictionary using sensor id as the key
    sensor_data[sensor] = df

for sensor, df in sensor_data.items():
    if 'delta' in df.columns:
        df.drop(columns=['delta'], inplace=True)
        df.drop(columns=['time'], inplace=True)
        df.drop(columns=['time_utc'], inplace=True)


# Example access:
print(sensor_data["Sensor_0057"].head())

   atm_pm_1_0  atm_pm_10_0  atm_pm_2_5  co2    hum   ma   temp   timestamp
0           3            6           6  400  36.79  6.1  26.99  1762611990
1           3            6           6  402  36.80  6.0  26.98  1762612000
2           3            6           5  403  36.81  5.9  26.99  1762612010
3           3            9           7  400  36.81  6.0  27.01  1762612020
4           4           11           7  404  36.81  6.2  27.00  1762612030
